<a href="https://colab.research.google.com/github/Int-Group-44/flowers-102/blob/main/flowers_102.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>